In [1]:
%pip install -q langchain-tavily langchain langchain-community langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import google.genai as genai
from langchain_tavily import TavilySearch
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv

from langchain.agents import initialize_agent, AgentExecutor, AgentType
from langchain.tools import Tool
from langchain_google_genai import ChatGoogleGenerativeAI
import os
load_dotenv()

True

In [3]:
tavily_search = TavilySearchResults(max_results=5)

In [4]:
tavily_search.invoke(
    input='Attention is all you need'
)

[{'title': 'Attention Is All You Need - Wikipedia',
  'url': 'https://en.wikipedia.org/wiki/Attention_Is_All_You_Need',
  'content': '"Attention Is All You Need"[1] is a 2017 landmark[2][3] research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer, based on the attention mechanism proposed in 2014 by Bahdanau et al.[4] It is considered a foundational[5] paper in modern artificial intelligence, as the transformer approach has become the main architecture of large language models like those based on GPT.[6][7] At the time, [...] ^ a b c d e f Vaswani, Ashish; Shazeer, Noam; Parmar, Niki; Uszkoreit, Jakob; Jones, Llion; Gomez, Aidan N; Kaiser, Łukasz; Polosukhin, Illia (December 2017). "Attention is All you Need". In I. Guyon and U. Von Luxburg and S. Bengio and H. Wallach and R. Fergus and S. Vishwanathan and R. Garnett (ed.). 31st Conference on Neural Information Processing Systems (N

In [5]:
tools = [
    Tool(name="TavilySearch", func=tavily_search.run, description="SEO Optimizer")
]

In [6]:
llm = ChatGoogleGenerativeAI(
    model='gemini-2.0-flash',
    max_retries=3,
    api_key=os.getenv('GEMINI_API_KEY')
)

In [7]:
def create_tavily_agent_prompt(content, additional_info=True):
    # search_prompt = f'''
    # You are an expert SEO Optimized post content writer and editor.
    # You have access to:
    # TavilySearch - for finding most relevant material on how to write SEO optimized posts.
    # The content of the post is:
    # {content}

    # Please make it SEO optimized.
    # If required, you can change wordings, but not the whole meaning.
    # Lastly, strictly only return the SEO Optimized post.
    # Please do not return any starting. No filler words and sentences. 
    # Just pure SEO Optimized Content. 
    # Return json of:
    # 'seo_optimized_post' : post
    # '''
    search_prompt = f'''
    You are an expert SEO-optimized post writer. Your task is to **rewrite** the given content in an SEO-friendly way.

    **Instructions:**
    - Improve readability, keyword placement, and engagement.
    - **DO NOT** return explanations, steps, or methods.
    - **Return ONLY** the optimized content in JSON format:
      ```json
      {{
          "seo_optimized_post": "Your fully optimized post here"
      }}
      ```

    **Original Post:**
    {content}
    '''

    if additional_info is not None:
        search_prompt = search_prompt + "\n"  f"Here are some additional details to consider : {additional_info}"
    
    return search_prompt

In [8]:
raw_content = f'''
**AI & ML: Level Up Your Mechanical Engineering Career! 🚀**

Hey #MechanicalEngineers! 👋 Ready to future-proof your skills?

AI and Machine Learning aren't just buzzwords anymore; they're revolutionizing manufacturing, design, and maintenance. Think:

*   **Predictive Maintenance:** Imagine algorithms that foresee equipment failures *before* they happen, saving time and $$$ 💰.
*   **Generative Design:** AI creating optimal designs we haven't even thought of yet – lighter, stronger, better! 🤯
*   **Automated Quality Control:** Catching defects with superhuman accuracy, ensuring top-notch products. ✅

**Why should YOU care?**

Knowing AI/ML will make you a *highly* sought-after engineer. It's about:

*   **Boosting Efficiency:** Streamlining processes, reducing waste. ♻️
*   **Driving Innovation:** Creating cutting-edge solutions. ✨
*   **Increasing Earning Potential:** Skills that are in demand pay BIG. 🤑

**Get started now:**

*   Explore online courses on platforms like Coursera or Udacity.
*   Start small – look for AI/ML applications within your current projects.
*   Join online communities and connect with experts!

#AIinManufacturing #MachineLearning #MechanicalEngineering #FutureofWork #EngineeringJobs #TechSkills #Innovation #CareerDevelopment
'''

In [12]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  
    verbose=True,
    # agent_kwargs={"prompt": create_tavily_agent_prompt(raw_content)},
    handle_parsing_errors=True
)

In [13]:
def seo_optimizer(platform, post):

    seo_prompt = f'''
    Use Tavily to search for the best SEO optimization techniques specific to [domain]. Analyze the top strategies, keyword usage, and content patterns from the search results. Based on these insights, identify key SEO elements (keywords, structure, engagement tactics) that improve {platform} visibility.

    Now, take my previous {platform} post:
    {post}

    Rewrite and transform it into an SEO-optimized {platform} post by applying the learned strategies. Ensure it remains engaging, professional, and aligned with {platform}'s algorithm best practices.

    Finally, only return the post and not any extra words.
    '''

    refined_response = agent.run(seo_prompt)
    return refined_response

In [14]:
print(seo_optimizer(platform='LinkedIn', post=raw_content))



> Entering new AgentExecutor chain...
I need to use Tavily to research SEO optimization techniques for LinkedIn, focusing on keywords, structure, and engagement. Then, I'll rewrite the provided LinkedIn post, incorporating these techniques to enhance its visibility and engagement.
Action: TavilySearch
Action Input: "SEO optimization techniques for LinkedIn posts keywords structure engagement"
Observation: [{'title': 'Peak Content Optimization: 7 Techniques for LinkedIn - 618Media', 'url': 'https://618media.com/en/blog/peak-content-optimization-for-linkedin/', 'content': 'Considering my own experience, making SEO methods a priority has made a huge difference in my LinkedIn content optimization.\n\nHave you considered using SEO techniques as part of your LinkedIn strategy?\n\nSEO techniques such as keyword research, optimized profiles, and structured posts improve discoverability and engagement on LinkedIn.\n\nMeasuring and Tracking LinkedIn Content Performance [...] A well-optimized p